In [1]:
import psi4
import forte
import forte.utils
import numpy as np

In [6]:
psi4.core.clean()
psi4.core.clean_options()
psi4.core.set_output_file('hf-1.5-mrpt3-relaxtwice.dat', False)

mol = psi4.geometry("""
H 0 0 0
F 0 0 1.5
symmetry c1
noreorient
""")

psi4.set_options({
    'basis': 'cc-pvdz',
    'restricted_docc':[2],
    'active':[4],
    'reference': 'rhf',
    'scf_type': 'direct'
})

forte_options = {
    'active_space_solver': 'fci',
    'correlation_solver': 'dsrg-mrpt3',
    'frozen_docc': [0],
    'restricted_docc': [2],
    'active': [4],
    'root_sym': 0,
    #'nroot': 4,
    'dsrg_s': 1.0,
    'relax_ref': 'twice',
    'maxiter': 100,
    'e_convergence': 1e-8,
    'semi_canonical': True
    }

Ecasci, wfn = psi4.energy('detci', return_wfn=True)
Edsrg = psi4.energy('forte', ref_wfn=wfn, forte_options=forte_options, return_wfn=True)


Error: option JOB_TYPE is not contained in the list of available options.

Did you mean? JOBTYPE SOS_TYPE


Error: option JOB_TYPE is not contained in the list of available options.

Did you mean? JOBTYPE SOS_TYPE



In [4]:
forte.ForteOptions().set_from_dict(forte_options)

RuntimeError: 
  ForteOptions::set_from_dict: option active_space_solver is not defined


In [14]:
ints_rdms = forte.utils.prepare_ints_rdms(wfn, {'frozen_docc': [0], 'restricted_docc': [2], 'active': [4]})
Eref = ints_rdms['reference_energy']
mo_space_info = ints_rdms['mo_space_info']
rdms = ints_rdms['rdms']
ints = ints_rdms['ints']
gamma1, lambda2, lambda3 = forte.spinorbital_cumulants(rdms)
eta1 = np.identity(gamma1.shape[0]) - gamma1

AttributeError: 'dict' object has no attribute 'get_str'

In [5]:
core = mo_space_info.corr_absolute_mo('RESTRICTED_DOCC')
actv = mo_space_info.corr_absolute_mo('ACTIVE')
virt = mo_space_info.corr_absolute_mo('RESTRICTED_UOCC')
hole = mo_space_info.corr_absolute_mo('GENERALIZED HOLE')
allm = mo_space_info.corr_absolute_mo('CORRELATED')

In [7]:
fock = forte.spinorbital_oei(ints,allm,allm)
_v = forte.spinorbital_tei(ints,allm,core,allm,core)
fock += np.einsum('piqi->pq', _v)
_v = forte.spinorbital_tei(ints,allm,actv,allm,actv)
fock += np.einsum('piqj,ij->pq', _v, gamma1) 

In [8]:
eri = forte.spinorbital_tei(ints, allm, allm, allm, allm)

In [9]:
np.save('fock.npy', fock)
np.save('eri.npy', eri)

In [10]:
fock.shape

(38, 38)